In [52]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from implementations import *
from helpers import *
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
DATA_TRAIN_PATH = 'train.csv'
y_or, tX_or, ids = load_csv_data(DATA_TRAIN_PATH)

In [54]:
y = y_or
tX = tX_or

In [55]:
tX = pd.DataFrame(tX)
tX.where(tX!=-999, inplace=True)
tX.fillna(tX.median(), inplace=True)    
tX = tX.values

In [56]:
y = y_for_logistic(y)

In [57]:
x_tr_tot, x_te_tot, y_tr_tot, y_te_tot = split_data(y,tX,0.7,1)

In [ ]:
initial_w = np.zeros(x_tr_tot.shape[1])
max_iters = 5000
gamma = 1e-9
w, loss = logistic_regression(y_tr_tot, x_tr_tot, initial_w, max_iters, gamma)

Current iteration=0, loss=121300.75659799043
Current iteration=500, loss=276737.1092019235
Current iteration=1000, loss=270105.5728942465
Current iteration=1500, loss=268183.1721119047
Current iteration=2000, loss=267456.65955464967
Current iteration=2500, loss=267117.08377504116
Current iteration=3000, loss=266935.5197234868


In [79]:
y_pred = predict_labels(w, x_te_tot, True)
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.72844
